Code cleanup

In [1]:
import h5py
import napari
import numpy as np
import pandas as pd
from abbott.h5_files import *
from abbott.conversions import *
from pathlib import Path
import nest_asyncio
nest_asyncio.apply()

In [6]:
print(h5_summary("Q:\carspi\ETiX-SBR-Cer1_exp2\ETiX-SBR_Cer11_s1.h5"))

filename:	ETiX-SBR_Cer11_s1.h5
condition:	NA

          name          shape  dtype   size element_size_um  img_type       stain cycle wavelength level
        /ch_00 (24, 512, 512) uint16  12.58 [3.0, 2.6, 2.6] intensity   Sox1-Cer1     0        488     1
        /ch_01 (24, 512, 512) uint16  12.58 [3.0, 2.6, 2.6] intensity         Bry     0        561     1
        /ch_02 (24, 512, 512) uint16  12.58 [3.0, 2.6, 2.6] intensity brightfield     0         NA     1
   /lbl_embryo (24, 512, 512)   bool   0.79 [3.0, 2.6, 2.6]     label          NA    NA         NA     1
/mask_filtered (24, 512, 512)  uint8   6.29 [3.0, 2.6, 2.6]     label          NA    NA         NA     1




In [3]:
fld = Path("Q:\carspi\ETiX-SBR-Cer1_exp2")

In [4]:
from skimage.measure import label, regionprops

def select_largest_object(bw_img):
    lbl_img = label(bw_img)
    props = regionprops(lbl_img)
    if not props: # sometimes we don't have ANY mask for a timepoint which would throw an error in the next step if we don't check for it
        return np.zeros(bw_img.shape)
    lbl = [p.label for p in props][np.argmax([p.area for p in props])]
    return (lbl_img == lbl).astype(np.uint8)

In [5]:
from scipy.ndimage import binary_fill_holes

out_name = 'mask_filtered'

for fn in fld.glob('*.h5'):
    print('processing {}...'.format(fn.name))

    with h5py.File(fn, 'r+') as f:
        lbl_in = h5_select(f, {'img_type': 'label', 'level': 1})[0]
        mask = lbl_in[...]
        timepoints = mask.shape[0]

        mask_filtered = []

        for i in range(timepoints):
            lbls = mask[i]
            lbl = select_largest_object(lbls) # select the largest label per timepoint
            lbl = binary_fill_holes(lbl) # fill holes in the mask
            mask_filtered.append(lbl)

        mask_filtered = [lbl.astype('uint8') for lbl in mask_filtered]
        mask_filtered = np.stack(mask_filtered, axis=0)

        #mask_filtered = np.stack(mask_filtered, axis = 0, dtype='uint8')

        if out_name in f.keys():
            del f[out_name]
        lbl_out = f.create_dataset(name= out_name, data= mask_filtered)
        h5_copy_attributes(lbl_in, lbl_out)
        

processing ETiX-SBR_Cer11_s1.h5...
processing ETiX-SBR_Cer11_s10.h5...
processing ETiX-SBR_Cer11_s100.h5...
processing ETiX-SBR_Cer11_s101.h5...
processing ETiX-SBR_Cer11_s102.h5...
processing ETiX-SBR_Cer11_s103.h5...
processing ETiX-SBR_Cer11_s104.h5...
processing ETiX-SBR_Cer11_s105.h5...
processing ETiX-SBR_Cer11_s106.h5...
processing ETiX-SBR_Cer11_s107.h5...
processing ETiX-SBR_Cer11_s108.h5...
processing ETiX-SBR_Cer11_s109.h5...
processing ETiX-SBR_Cer11_s11.h5...
processing ETiX-SBR_Cer11_s110.h5...
processing ETiX-SBR_Cer11_s111.h5...
processing ETiX-SBR_Cer11_s112.h5...
processing ETiX-SBR_Cer11_s113.h5...
processing ETiX-SBR_Cer11_s114.h5...
processing ETiX-SBR_Cer11_s115.h5...
processing ETiX-SBR_Cer11_s116.h5...
processing ETiX-SBR_Cer11_s117.h5...
processing ETiX-SBR_Cer11_s118.h5...
processing ETiX-SBR_Cer11_s119.h5...
processing ETiX-SBR_Cer11_s12.h5...
processing ETiX-SBR_Cer11_s120.h5...
processing ETiX-SBR_Cer11_s121.h5...
processing ETiX-SBR_Cer11_s122.h5...
proces

In [5]:
print(h5_summary("Q:\carspi\ETiX-SBR-Cer1_exp1\ETiX-Cer1-SBR1_s1.h5"))

filename:	ETiX-Cer1-SBR1_s1.h5
condition:	NA

          name          shape  dtype   size   element_size_um  img_type       stain cycle wavelength level
      /ch_00/1 (23, 512, 512) uint16  12.06   [3.0, 2.6, 2.6] intensity brightfield     0         NA     1
      /ch_00/2 (23, 256, 256) uint16   3.01   [3.0, 5.2, 5.2] intensity brightfield     0         NA     2
      /ch_00/3  (7, 128, 128) uint16   0.23 [9.0, 10.4, 10.4] intensity brightfield     0         NA     3
      /ch_01/1 (23, 512, 512) uint16  12.06   [3.0, 2.6, 2.6] intensity   Sox1_Cer1     0        488     1
      /ch_01/2 (23, 256, 256) uint16   3.01   [3.0, 5.2, 5.2] intensity   Sox1_Cer1     0        488     2
      /ch_01/3  (7, 128, 128) uint16   0.23 [9.0, 10.4, 10.4] intensity   Sox1_Cer1     0        488     3
      /ch_02/1 (23, 512, 512) uint16  12.06   [3.0, 2.6, 2.6] intensity         Bry     0        561     1
      /ch_02/2 (23, 256, 256) uint16   3.01   [3.0, 5.2, 5.2] intensity         Bry     0        5

In [ ]:
import h5py
import napari

fn = "Q:\carspi\ETiX-SBR-test\ETiX-Cer1-SBR1_s10.h5"

with h5py.File(fn, 'r') as f:

   
    for channel_name in f.keys():
        bf = f['ch_00/1'][...]
        mask = f['embryo_mask/1'][...]
        filtered = f['mask_filtered'][...]


viewer = napari.Viewer()        
viewer.add_image(bf)
viewer.add_labels(filtered)
viewer.add_labels(mask)

